In [1]:
import os

import cntk
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

from scipy.io import wavfile
import numpy as np
from random import shuffle
import math

Using CNTK backend
C:\Users\Chanok Pathompatai\Anaconda3\envs\cntk-py36\lib\site-packages\keras\backend\cntk_backend.py:25: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


## FRAMEWORK

In [2]:
cntk.try_set_default_device( cntk.all_devices()[0] )

RuntimeError: Process wide default device cannot be changed since it has been frozen by being implicitly used as the default device in a CNTK API call; Current default = CPU, New default = GPU[0] GeForce GTX 1050.

[CALL STACK]
    > std::enable_shared_from_this<Microsoft::MSR::CNTK::MatrixBase>::  operator=
    - CNTK::DeviceDescriptor::  TrySetDefaultDevice
    - PyInit__cntk_py (x2)
    - PyCFunction_FastCallDict
    - PyObject_CallFunctionObjArgs
    - PyEval_EvalFrameDefault
    - Py_CheckFunctionResult
    - PyObject_CallFunctionObjArgs
    - PyEval_EvalFrameDefault
    - Py_CheckFunctionResult
    - PyEval_GetGlobals (x2)
    - PyCFunction_FastCallDict
    - PyObject_CallFunctionObjArgs
    - PyEval_EvalFrameDefault



## DATA PREPARATION

In [4]:
sec = 3

root, _dirs, files = next(os.walk( os.path.join( os.getcwd(), os.path.join( "dataset", "train" ))))
train_paths = [ os.path.join( root, file ) for file in files ]

root, _dirs, files = next(os.walk( os.path.join( os.getcwd(), os.path.join( "dataset", "test" ))))
test_paths = [ os.path.join( root, file ) for file in files ]

In [8]:
def create_samples(file_path, sec=1):
    rate, audio = wavfile.read( file_path )
    rate *= sec
    audio = np.mean(audio, axis=1)
    audio = np.trim_zeros(audio)
    audio = audio[:len(audio)-(len(audio)%rate)]
    rates = [ audio[i:i+rate] for i in range( 0, len(audio), rate ) ]
    return rates

In [5]:
def get_samples(paths, sec=1):
    samples = []
    for file_path in paths:
        rates = create_samples(file_path=file_path, sec=sec)
        if "ปกติ" in file_path:
            label = 0
        else:
            label = 1
        for frame in rates:
            samples.append( (frame, label) )
    shuffle(samples)
    return samples

In [6]:
samples = get_samples( train_paths, sec )
x_train = np.asarray( [sample[0] for sample in samples] ,dtype=np.float32)
x_train = np.tanh( x_train )
# x_train = np.tan( x_train )
y_train = keras.utils.to_categorical(np.asarray( [sample[1] for sample in samples] ), num_classes=2)

In [ ]:
x_train.shape

## MODEL

In [7]:
def get_logistic_model(input_dim):
    
    model = Sequential()
    
    model.add(Dense(256, input_dim=input_dim))
    model.add(Activation('linear'))
#     model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(Activation('linear'))
#     model.add(Dropout(0.5))
#     model.add(Dense(32))
#     model.add(Activation('linear'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
    return model

In [8]:
model = get_logistic_model( x_train.shape[1] )
model.fit( x=x_train, y=y_train, epochs=50, batch_size=200 )

Epoch 1/50
357/357 [==============================] - 0s 1ms/step - loss: 1.7328 - acc: 0.5238
Epoch 2/50
357/357 [==============================] - 0s 819us/step - loss: 0.3588 - acc: 0.8571
Epoch 3/50
357/357 [==============================] - 0s 807us/step - loss: 0.1464 - acc: 0.9468
Epoch 4/50
357/357 [==============================] - 0s 807us/step - loss: 0.1344 - acc: 0.9664
Epoch 5/50
357/357 [==============================] - 0s 810us/step - loss: 0.0057 - acc: 1.0000
Epoch 6/50
357/357 [==============================] - 0s 818us/step - loss: 0.0294 - acc: 0.9888
Epoch 7/50
357/357 [==============================] - 0s 807us/step - loss: 0.0131 - acc: 0.9944
Epoch 8/50
357/357 [==============================] - 0s 810us/step - loss: 0.0048 - acc: 1.0000
Epoch 9/50
357/357 [==============================] - 0s 812us/step - loss: 0.0020 - acc: 1.0000
Epoch 10/50
357/357 [==============================] - 0s 806us/step - loss: 8.7328e-04 - acc: 1.0000
Epoch 11/50
357/357 [======

## Evaluate

In [9]:
def evalualte(file_path, sec=1):
    print( "FILE: %s => " % file_path, end="" )
    rate, audio = wavfile.read( file_path )
    rate*=sec
    audio = np.mean(audio, axis=1)
    audio = np.trim_zeros(audio)
    audio = audio[:len(audio)-(len(audio)%rate)]
    rates = [ audio[i:i+rate] for i in range( 0, len(audio), rate ) ]

    x_test = np.asarray( [rate for rate in rates], dtype=np.float32 )
    x_test = np.tanh( x_test )
#     x_test = np.tan( x_test )

    label = 1
    if "ปกติ" in file_path:
        label = 0
    results = model.predict( x_test )
    count_label = 0
    for result in results:
        i = np.argmax(result)
        if i == label:
            count_label += 1
            
#         if i == 0:
#             print("ปกติ")
#         else:
#             print("เศร้า")
        
    count_wrong = len( x_test ) - count_label
    
    if count_wrong > count_label:
        print("WRONG")
        return 0
    if count_wrong == count_label:
        print("Do not know")
        return 0
    print("RIGHT")
    return 1

In [10]:
count = 0
shuffle(test_paths)
for path in test_paths:
    count += evalualte(path, sec)
s_acc = "%.2f" % (count / len(test_paths) * 100 )
print( "Acc = %s %%" % s_acc )

FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า08.wav => Do not know
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า16.wav => RIGHT
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า18.wav => RIGHT
FILE: D:\Documents\Works\Dreparion\dataset\test\ปกติ59.wav => WRONG
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า11.wav => Do not know
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า07.wav => Do not know
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า13.wav => RIGHT
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า21.wav => WRONG
FILE: D:\Documents\Works\Dreparion\dataset\test\ปกติ64.wav => RIGHT
FILE: D:\Documents\Works\Dreparion\dataset\test\ปกติ61.wav => WRONG
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า20.wav => Do not know
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า19.wav => RIGHT
FILE: D:\Documents\Works\Dreparion\dataset\test\เศร้า15.wav => Do not know
FILE: D:\Documents\Works\Dreparion\dataset\test\ปกติ56.wav => WRONG
FILE: D:

In [11]:
model_folder = os.path.join( os.getcwd(), "Model" )
_, _, files = next( os.walk(model_folder) )

model_name = "Model_" + str(len(files)+1).zfill(4) + ".h5"
model.save( os.path.join( model_folder, model_name ) )
 
with open( 'log.txt', 'a' ) as logfile:
    logfile.write( "%s : %s\n" % (model_name, s_acc) )

In [7]:
 create_samples, sec=1)

NameError: name 'file_path' is not defined

In [ ]:
create_samples(file_path, sec=1)